In [1]:
import numpy as np

# Load the parity check matrix of the classical code that is used to construct the hgp code
# h = np.loadtxt('/home/normaluser/ck/linkequantum-qLDPC/quits/parity_check_matrices/n=12_dv=3_dc=4_dist=6.txt', dtype=int)
# h = np.loadtxt('/home/normaluser/ck/linkequantum-qLDPC/quits/parity_check_matrices/n=20_dv=3_dc=4_dist=8_arXiv2311_03307.txt', dtype=int)
# h = np.loadtxt('/home/normaluser/ck/linkequantum-qLDPC/quits/parity_check_matrices/n=24_dv=3_dc=4_dist=10_arXiv2311_03307.txt', dtype=int)
h = np.loadtxt('/home/normaluser/ck/linkequantum-qLDPC/quits/parity_check_matrices/n=28_dv=3_dc=4_dist=10.txt', dtype=int)

print('Shape of classical code\'s parity check matrix: ', h.shape)

Shape of classical code's parity check matrix:  (21, 28)


In [2]:
from quits.qldpc_code import *

code = HgpCode(h, h)         # Define the HgpCode object
code.build_graph(seed=22)    # Build the Tanner graph and assign directions to its edges. 
                             # For this specific h, seed=22 gives a circuit with entangling depth 8. 
num_zcheck, num_data = code.hz.shape
num_xcheck, num_data = code.hx.shape
num_logical = code.lz.shape[0]
depth = sum(list(code.num_colors.values())) 
print('# data qubits: ', num_data, ' # logical qubits: ', num_logical)
print('# z-check qubits: ', num_zcheck, ' # x-check qubits: ', num_xcheck)
print('# layers of entangling gates: ', depth)

print("---------------------------------------------------")

print("比特数目: ", num_data + num_zcheck + num_xcheck)

import stim

from quits.circuit import get_qldpc_mem_circuit, check_overlapping_CX

p = 1e-3           # physical error rate
num_rounds = 0    # number of rounds (T-1)
basis = 'X'        # 'Z' or 'X'

idle_error = p     # Float (DEPOLARIZE1 error with rate p) or tuple of 3 floats (px, py, pz)
sqgate_error = p   # Float (DEPOLARIZE1 error with rate p) or tuple of 3 floats (px, py, pz)
tqgate_error = p   # Float (DEPOLARIZE2 error with rate p) or tuple of 15 floats (pix, ..., pzz)
spam_error = p     # Float

# Generate the memory experiment circuit
circuit = stim.Circuit(get_qldpc_mem_circuit(code, idle_error, sqgate_error, tqgate_error, spam_error, num_rounds, basis=basis))
check_overlapping_CX(circuit)    # Check for overlapping CX gates in the same layer. Nothing should be printed.

print(circuit.without_noise())

file_name = f"hgp_code_n_{num_data + num_zcheck + num_xcheck}_k_{num_logical}_r_1_{basis}"

# data qubits:  1225  # logical qubits:  49
# z-check qubits:  588  # x-check qubits:  588
# layers of entangling gates:  12
---------------------------------------------------
比特数目:  2401
RX 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 

In [3]:
no_noise_circuit = circuit.without_noise()

保存含噪声的Stim操作，用于后续测量。

In [4]:
import stim
import os

def save_stim_circuit(stim_str: str, folder_path: str, filename: str = "circuit.stim"):
    """
    将Stim电路字符串转换为Stim对象，并保存为.stim文件到指定文件夹。
    
    Args:
        stim_str (str): Stim电路的字符串表示。
        folder_path (str): 保存文件的文件夹路径。
        filename (str): 文件名，默认为"circuit.stim"。
    """
    # 确保文件夹存在
    os.makedirs(folder_path, exist_ok=True)
    
    if not isinstance(stim_str, str):
        # raise ValueError("stim_str must be a string representing a Stim circuit.")
        stim_str = str(stim_str)

    # 从字符串创建Stim电路
    circuit = stim.Circuit(stim_str)
    
    # 构建完整路径
    full_path = os.path.join(folder_path, filename)
    
    # 保存到文件
    circuit.to_file(full_path)
    
    print(f"Stim circuit saved to {full_path}")

# 保存Stim电路到指定文件夹
output_folder = '/home/normaluser/ck/linkequantum-qLDPC/qLDPC/qldpc_stim_circuit'
# fiilename = "hgp_code_n_441_k_9_r_1_code.stim"
save_stim_circuit(circuit, output_folder, file_name + ".stim")

Stim circuit saved to /home/normaluser/ck/linkequantum-qLDPC/qLDPC/qldpc_stim_circuit/hgp_code_n_2401_k_49_r_1_X.stim


### 转换为QASM，同时进行一定的操作

OpenQASM不支持噪声。在2.0版本中，不支持对于经典寄存器数据的操作。（不支持Detector和OBSERVABLE_INCLUDE）

In [5]:
qasm = no_noise_circuit.to_qasm(open_qasm_version = 2, skip_dets_and_obs = True)

print(qasm.strip().replace('\n\n', '\n'))

OPENQASM 2.0;
include "qelib1.inc";
qreg q[2401];
creg rec[2401];
reset q[0]; h q[0]; // decomposed RX
reset q[1]; h q[1]; // decomposed RX
reset q[2]; h q[2]; // decomposed RX
reset q[3]; h q[3]; // decomposed RX
reset q[4]; h q[4]; // decomposed RX
reset q[5]; h q[5]; // decomposed RX
reset q[6]; h q[6]; // decomposed RX
reset q[7]; h q[7]; // decomposed RX
reset q[8]; h q[8]; // decomposed RX
reset q[9]; h q[9]; // decomposed RX
reset q[10]; h q[10]; // decomposed RX
reset q[11]; h q[11]; // decomposed RX
reset q[12]; h q[12]; // decomposed RX
reset q[13]; h q[13]; // decomposed RX
reset q[14]; h q[14]; // decomposed RX
reset q[15]; h q[15]; // decomposed RX
reset q[16]; h q[16]; // decomposed RX
reset q[17]; h q[17]; // decomposed RX
reset q[18]; h q[18]; // decomposed RX
reset q[19]; h q[19]; // decomposed RX
reset q[20]; h q[20]; // decomposed RX
reset q[21]; h q[21]; // decomposed RX
reset q[22]; h q[22]; // decomposed RX
reset q[23]; h q[23]; // decomposed RX
reset q[24]; h q[2

In [6]:
import re

def process_qasm(qasm):
    """
    Process the QASM string to reorganize decomposed MR (measure-reset) blocks.
    Groups all measure operations first, followed by a barrier, then all reset operations, followed by another barrier.
    Retains all other content in the QASM string.
    
    Args:
        qasm (str): The input QASM string.
    
    Returns:
        str: The processed QASM string with MR blocks reorganized.
    """
    lines = qasm.split('\n')
    new_lines = []
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if re.match(r'measure q\[\d+\] -> rec\[\d+\]; reset q\[\d+\]; // decomposed MR', line):
            # Start of MR block
            measures = []
            resets = []
            while i < len(lines) and re.match(r'measure q\[\d+\] -> rec\[\d+\]; reset q\[\d+\]; // decomposed MR', lines[i].strip()):
                match = re.match(r'(measure q\[\d+\] -> rec\[\d+\];) (reset q\[\d+\];) // decomposed MR', lines[i].strip())
                measures.append(match.group(1))
                resets.append(match.group(2))
                i += 1
            # Check if next is barrier q;
            if i < len(lines) and lines[i].strip() == 'barrier q;':
                new_lines.extend(measures)
                new_lines.append('barrier q;')
                new_lines.extend(resets)
                new_lines.append('barrier q;')
                i += 1  # Skip the original barrier
            else:
                # If no barrier, just add back as is
                new_lines.extend([m + ' ' + r + ' // decomposed MR' for m, r in zip(measures, resets)])
        else:
            new_lines.append(lines[i])
            i += 1
    return '\n'.join(new_lines)

# Example usage:
# processed_qasm = process_qasm(qasm)
# Then, you can save or print processed_qasm as needed.

In [7]:
right_qasm = process_qasm(qasm)

print(right_qasm.strip().replace('\n\n', '\n'))

OPENQASM 2.0;
include "qelib1.inc";
qreg q[2401];
creg rec[2401];
reset q[0]; h q[0]; // decomposed RX
reset q[1]; h q[1]; // decomposed RX
reset q[2]; h q[2]; // decomposed RX
reset q[3]; h q[3]; // decomposed RX
reset q[4]; h q[4]; // decomposed RX
reset q[5]; h q[5]; // decomposed RX
reset q[6]; h q[6]; // decomposed RX
reset q[7]; h q[7]; // decomposed RX
reset q[8]; h q[8]; // decomposed RX
reset q[9]; h q[9]; // decomposed RX
reset q[10]; h q[10]; // decomposed RX
reset q[11]; h q[11]; // decomposed RX
reset q[12]; h q[12]; // decomposed RX
reset q[13]; h q[13]; // decomposed RX
reset q[14]; h q[14]; // decomposed RX
reset q[15]; h q[15]; // decomposed RX
reset q[16]; h q[16]; // decomposed RX
reset q[17]; h q[17]; // decomposed RX
reset q[18]; h q[18]; // decomposed RX
reset q[19]; h q[19]; // decomposed RX
reset q[20]; h q[20]; // decomposed RX
reset q[21]; h q[21]; // decomposed RX
reset q[22]; h q[22]; // decomposed RX
reset q[23]; h q[23]; // decomposed RX
reset q[24]; h q[2

In [8]:
# Add this new cell after the existing process_qasm function cell

import re

def process_mx_qasm(qasm):
    """
    Process the QASM string to reorganize decomposed MX (measure X) blocks.
    Groups all initial H gates first, followed by all measure operations.
    Removes the final H gates in each MX block.
    Retains all other content in the QASM string.
    
    Args:
        qasm (str): The input QASM string.
    
    Returns:
        str: The processed QASM string with MX blocks reorganized.
    """
    lines = qasm.split('\n')
    new_lines = []
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if re.match(r'h q\[\d+\]; measure q\[\d+\] -> rec\[\d+\]; h q\[\d+\]; // decomposed MX', line):
            # Start of MX block
            initial_hs = []
            measures = []
            while i < len(lines) and re.match(r'h q\[\d+\]; measure q\[\d+\] -> rec\[\d+\]; h q\[\d+\]; // decomposed MX', lines[i].strip()):
                match = re.match(r'(h q\[\d+\];) (measure q\[\d+\] -> rec\[\d+\];) (h q\[\d+\];) // decomposed MX', lines[i].strip())
                initial_hs.append(match.group(1))
                measures.append(match.group(2))
                # Ignore the final h
                i += 1
            # Add all initial h's first
            new_lines.extend(initial_hs)
            # Then all measures
            new_lines.extend(measures)
        else:
            new_lines.append(lines[i])
            i += 1
    return '\n'.join(new_lines)

# Example usage:
# processed_qasm = process_mx_qasm(qasm)
# Then, you can save or print processed_qasm as needed.

In [9]:
right_qasm = process_mx_qasm(right_qasm)

print(right_qasm.strip().replace('\n\n', '\n'))

OPENQASM 2.0;
include "qelib1.inc";
qreg q[2401];
creg rec[2401];
reset q[0]; h q[0]; // decomposed RX
reset q[1]; h q[1]; // decomposed RX
reset q[2]; h q[2]; // decomposed RX
reset q[3]; h q[3]; // decomposed RX
reset q[4]; h q[4]; // decomposed RX
reset q[5]; h q[5]; // decomposed RX
reset q[6]; h q[6]; // decomposed RX
reset q[7]; h q[7]; // decomposed RX
reset q[8]; h q[8]; // decomposed RX
reset q[9]; h q[9]; // decomposed RX
reset q[10]; h q[10]; // decomposed RX
reset q[11]; h q[11]; // decomposed RX
reset q[12]; h q[12]; // decomposed RX
reset q[13]; h q[13]; // decomposed RX
reset q[14]; h q[14]; // decomposed RX
reset q[15]; h q[15]; // decomposed RX
reset q[16]; h q[16]; // decomposed RX
reset q[17]; h q[17]; // decomposed RX
reset q[18]; h q[18]; // decomposed RX
reset q[19]; h q[19]; // decomposed RX
reset q[20]; h q[20]; // decomposed RX
reset q[21]; h q[21]; // decomposed RX
reset q[22]; h q[22]; // decomposed RX
reset q[23]; h q[23]; // decomposed RX
reset q[24]; h q[2

删除其中的reset操作

In [10]:
def remove_resets(qasm):
    """
    Remove reset operations from the QASM string.
    Assumes the QASM has been processed by process_qasm, where resets are grouped together after measures.
    
    Args:
        qasm (str): The input QASM string.
    
    Returns:
        str: The QASM string with reset operations removed.
    """
    lines = qasm.split('\n')
    new_lines = []
    skip = False
    for line in lines:
        stripped = line.strip()
        if stripped.startswith('reset q['):
            skip = True  # Start skipping reset lines
        elif skip and stripped == 'barrier q;':
            skip = False  # Stop skipping after the barrier following resets
            continue  # Skip this barrier as well
        elif not skip:
            new_lines.append(line)
    return '\n'.join(new_lines)

# Example usage:
qasm_output_cx = remove_resets(right_qasm)
# Then, you can save or print final_qasm as needed.
print(qasm_output_cx.strip().replace('\n\n', '\n'))

OPENQASM 2.0;
include "qelib1.inc";
qreg q[2401];
creg rec[2401];

h q[28];
h q[29];
h q[30];
h q[31];
h q[32];
h q[33];
h q[34];
h q[35];
h q[36];
h q[37];
h q[38];
h q[39];
h q[40];
h q[41];
h q[42];
h q[43];
h q[44];
h q[45];
h q[46];
h q[47];
h q[48];
h q[77];
h q[78];
h q[79];
h q[80];
h q[81];
h q[82];
h q[83];
h q[84];
h q[85];
h q[86];
h q[87];
h q[88];
h q[89];
h q[90];
h q[91];
h q[92];
h q[93];
h q[94];
h q[95];
h q[96];
h q[97];
h q[126];
h q[127];
h q[128];
h q[129];
h q[130];
h q[131];
h q[132];
h q[133];
h q[134];
h q[135];
h q[136];
h q[137];
h q[138];
h q[139];
h q[140];
h q[141];
h q[142];
h q[143];
h q[144];
h q[145];
h q[146];
h q[175];
h q[176];
h q[177];
h q[178];
h q[179];
h q[180];
h q[181];
h q[182];
h q[183];
h q[184];
h q[185];
h q[186];
h q[187];
h q[188];
h q[189];
h q[190];
h q[191];
h q[192];
h q[193];
h q[194];
h q[195];
h q[224];
h q[225];
h q[226];
h q[227];
h q[228];
h q[229];
h q[230];
h q[231];
h q[232];
h q[233];
h q[234];
h q[235];
h q[236];
h q[2

将CX门转换为CZ门

In [11]:
from qiskit.circuit import QuantumCircuit, QuantumRegister
from qiskit.compiler import transpile
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.circuit.library import HGate, CZGate

# 原始 QASM 字符串输入
qasm_input = """
OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg c[2];
h q[0];
barrier q;
cx q[1],q[0];
measure q[0] -> c[0];
reset q[1];
"""

def remove_barrier_from_qasm(qasm_str: str) -> str:
    lines = qasm_str.splitlines()
    new_lines = [
        line for line in lines
        if not line.strip().startswith("barrier")
    ]
    return "\n".join(new_lines)


# 目标基准门集
target_basis_gates = ['h', 'cz', 'measure', 'reset']
# 移除 barrier
qc_original_without_barrier_qasm = remove_barrier_from_qasm(qasm_input)

# --- Step 1: 读取 QASM 字符串 ---
qc_original = QuantumCircuit.from_qasm_str(qc_original_without_barrier_qasm)

qc_final =  transpile(qc_original, basis_gates=target_basis_gates,
                                        optimization_level=2,
                                        seed_transpiler=0)

import qiskit.qasm2
# --- Step 3: 输出新的 QASM 字符串 ---
qasm_output = qiskit.qasm2.dumps(qc_final)

print("--- 转换后的 QASM 线路 (Qiskit 1.2.4) ---")
print(qasm_output)

print("\n--- 门计数验证 ---")
print("原始线路门:", qc_original.count_ops())
print("转换后线路门:", qc_final.count_ops())

--- 转换后的 QASM 线路 (Qiskit 1.2.4) ---
OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg c[2];
cz q[1],q[0];
h q[0];
reset q[1];
measure q[0] -> c[0];

--- 门计数验证 ---
原始线路门: OrderedDict([('h', 1), ('cx', 1), ('measure', 1), ('reset', 1)])
转换后线路门: OrderedDict([('cz', 1), ('h', 1), ('reset', 1), ('measure', 1)])


In [12]:
from qiskit.circuit import QuantumCircuit, QuantumRegister
from qiskit.compiler import transpile
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.circuit.library import HGate, CZGate

# 原始 QASM 字符串输入
qasm_input = qasm_output_cx

# 目标基准门集
target_basis_gates = ['h', 'cz', 'measure', 'reset']

qc_original_without_barrier_qasm = remove_barrier_from_qasm(qasm_input)

# --- Step 1: 读取 QASM 字符串 ---
qc_original = QuantumCircuit.from_qasm_str(qc_original_without_barrier_qasm)
 # 移除所有 barrier 操作

qc_final =  transpile(qc_original, basis_gates=target_basis_gates,
                                        optimization_level=3,
                                        seed_transpiler=0)

import qiskit.qasm2
# --- Step 3: 输出新的 QASM 字符串 ---
qasm_output_cz = qiskit.qasm2.dumps(qc_final)

print("--- 转换后的 QASM 线路 (Qiskit 1.2.4) ---")
print(qasm_output_cz)

print("\n--- 门计数验证 ---")
print("原始线路门:", qc_original.count_ops())
print("转换后线路门:", qc_final.count_ops())

--- 转换后的 QASM 线路 (Qiskit 1.2.4) ---
OPENQASM 2.0;
include "qelib1.inc";
qreg q[2401];
creg rec[2401];
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];
h q[7];
h q[8];
h q[9];
h q[10];
h q[11];
h q[12];
h q[13];
h q[14];
h q[15];
h q[16];
h q[17];
h q[18];
h q[19];
h q[20];
h q[21];
h q[22];
h q[23];
h q[24];
h q[25];
h q[26];
h q[27];
h q[28];
cz q[28],q[13];
h q[29];
cz q[29],q[23];
cz q[29],q[4];
h q[4];
h q[30];
cz q[30],q[22];
cz q[30],q[14];
h q[14];
h q[31];
cz q[31],q[10];
cz q[31],q[27];
h q[10];
h q[27];
h q[32];
cz q[32],q[2];
h q[33];
cz q[33],q[6];
cz q[33],q[12];
h q[12];
h q[34];
cz q[34],q[25];
cz q[34],q[23];
h q[35];
cz q[35],q[15];
cz q[35],q[7];
h q[7];
h q[36];
cz q[36],q[20];
h q[20];
h q[37];
cz q[37],q[5];
cz q[37],q[16];
h q[5];
h q[16];
h q[38];
cz q[38],q[26];
cz q[38],q[23];
h q[26];
h q[23];
h q[39];
cz q[39],q[18];
cz q[39],q[15];
h q[18];
h q[15];
h q[40];
cz q[40],q[0];
cz q[40],q[22];
h q[0];
h q[22];
h q[41];
cz q[41],q[21];
cz q[41],q[2];
h q[2]

转换为qasm文件

In [13]:
import os
target_directory = "/home/normaluser/ck/linkequantum-qLDPC/qLDPC/qldpc_qasm_circuit"
# file_name = f"hgp_code_n_441_k_9_r_1_cz.qasm"
# file_name = f"hgp_code_n_1225_k_25_r_1_cz.qasm"

# 2. 构造完整的文件路径
full_file_path = os.path.join(target_directory, file_name + "_cz.qasm")

# 3. 确保目标文件夹存在
try:
    # 递归创建目录。如果目录已存在，不会抛出错误 (exist_ok=True)。
    os.makedirs(target_directory, exist_ok=True)
    print("目标文件夹已准备就绪。")
except OSError as e:
    print(f"创建目标文件夹时发生错误: {e}")
    # 如果文件夹创建失败（例如，权限不足），则停止执行
    exit()


# 4. 将 QASM 字符串写入文件
try:
    # 'w' 模式: 写入 (如果文件不存在则创建，如果存在则覆盖)
    with open(full_file_path, 'w', encoding='utf-8') as f:
        f.write(qasm_output_cz)
    
    print(f"\n✅ 成功将 QASM 代码保存到: {full_file_path}")

except IOError as e:
    print(f"\n❌ 保存文件时发生 I/O 错误: {e}")
except Exception as e:
    print(f"\n❌ 发生意外错误: {e}")

目标文件夹已准备就绪。

✅ 成功将 QASM 代码保存到: /home/normaluser/ck/linkequantum-qLDPC/qLDPC/qldpc_qasm_circuit/hgp_code_n_2401_k_49_r_1_X_cz.qasm
